In [ ]:
pip install ultralytics

In [4]:
from ultralytics import YOLO
import supervision as sv
import cv2


In [5]:
model = YOLO("yolov9c")

# Define annotators
ellipse_annotator = sv.EllipseAnnotator()
triangle_annotator = sv.TriangleAnnotator()

class DetectandAnnotate():
    def __init__(self, input_file_path, output_file_path):
        self.input_file_path = input_file_path
        self.output_file_path = output_file_path
        self.cv_height = cv2.CAP_PROP_FRAME_HEIGHT
        self.cv_width = cv2.CAP_PROP_FRAME_WIDTH
        self.cv_fps = cv2.CAP_PROP_FPS

    def detect_and_annotate(self):
        input = cv2.VideoCapture(self.input_file_path)
        width, height, fps = (int(input.get(x)) for x in (self.cv_width, self.cv_height, self.cv_fps))
        output = cv2.VideoWriter(self.output_file_path, cv2.VideoWriter.fourcc(*"mp4v"), fps, (width, height))

        while input.isOpened():
            options, frame = input.read()
            if not options:
                break

            results = model.predict(frame)
            detections = sv.Detections.from_ultralytics(results[0])

            annotated_frame = ellipse_annotator.annotate(
                scene=frame.copy(),
                detections=detections
            )

            annotated_frame = triangle_annotator.annotate(
                scene=annotated_frame,
                detections=detections
            )

            output.write(annotated_frame)
            cv2.imshow("annotated_frame", annotated_frame)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
    
        input.release()
        output.release()
        cv2.destroyAllWindows()
        

100%|██████████| 49.4M/49.4M [00:37<00:00, 1.37MB/s]


In [7]:
if __name__ == "__main__":
    input_path = "input_videos/videoplayback.mp4"                # path to your input video
    output_path = "output_videos/annotated_output.mp4"            # path to save annotated video

    detector = DetectandAnnotate(input_path, output_path)
    detector.detect_and_annotate()